- id : 샘플 아이디
- Gender : 전복 성별
- Lenght : 전복 길이
- Diameter : 전복 둘레
- Height : 전복 키 
- Whole : Weight : 전복 전체 무게
- Shucked Weight : 껍질을 제외한 무게
- Viscra Weight : 내장 무게
- Shell Weight : 껍질 무게
- Target : 전복 나이

In [300]:
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings("ignore")

In [306]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [308]:
train['Water Weight'] = train['Shucked Weight'] - train['Viscra Weight']
test['Water Weight'] = test['Shucked Weight'] - test['Viscra Weight']

In [309]:
train.drop([130, 847, 1035, 435], axis = 0, inplace = True)

In [348]:
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from ngboost import NGBRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.metrics import mean_absolute_error

In [311]:
train['Gender'] = train['Gender'].map({'F' : 2, 'M' : 1, 'I' : 0})
test['Gender'] = test['Gender'].map({'F' : 2, 'M' : 1, 'I' : 0})

In [312]:
train['heavy_weight'] = [list(i).index(i.max()) for i in train[['Viscra Weight', 'Shell Weight', 'Water Weight']].values]
test['heavy_weight'] = [list(i).index(i.max()) for i in test[['Viscra Weight', 'Shell Weight', 'Water Weight']].values]

In [333]:
X = train.drop(['id', 'Shucked Weight', 'Target'], axis = 1)
y = np.log1p(train['Target'])

target = test[X.columns]

In [334]:
X['Viscra Weight'] /= X['Whole Weight']
X['Water Weight'] /= X['Whole Weight']
X['Shell Weight'] /= X['Whole Weight']

In [335]:
target['Viscra Weight'] /= target['Whole Weight']
target['Water Weight'] /= target['Whole Weight']
target['Shell Weight'] /= target['Whole Weight']

In [338]:
X.drop('Whole Weight', axis = 1, inplace = True)
target.drop('Whole Weight', axis = 1, inplace = True)

In [339]:
def get_target_group(x) :
    if x <= 7 :
        return 1
    elif 7 < x <= 11 :
        return 2
    elif 12 < x < 20 :
        return 3
    else :
        return 4

In [340]:
def NMAE(true, pred) -> float:
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

In [342]:
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

In [343]:
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

In [344]:
cb_pred = np.zeros(target.shape[0])
cb_nmae = []
for i, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    tr_data = Pool(data = tr_x, label = tr_y, cat_features = ['Gender', 'heavy_weight'])
    val_data = Pool(data = val_x, label = val_y, cat_features = ['Gender', 'heavy_weight'])
    
    cb = CatBoostRegressor(random_state = 42, iterations = 2000, depth = 4, learning_rate = 0.02, cat_features = ['Gender', 'heavy_weight'], loss_function = 'MAE')
    cb.fit(tr_data, eval_set = val_data, early_stopping_rounds = 300, verbose = 0)
    
    val_pred = np.expm1(cb.predict(val_data))
    val_nmae = NMAE(np.expm1(val_y), val_pred)
    cb_nmae.append(val_nmae)
    print(f'{i + 1} Fold NMAE = {val_nmae}')
    
    fold_pred = np.expm1(cb.predict(target)) / kf.n_splits
    cb_pred += fold_pred
print(f'\n avg = {np.mean(cb_nmae)}')

1 Fold NMAE = 0.1459782746734867
2 Fold NMAE = 0.14187409359808087
3 Fold NMAE = 0.1523499628922037
4 Fold NMAE = 0.1420125043874067
5 Fold NMAE = 0.1311080822811978
6 Fold NMAE = 0.17031385705263266
7 Fold NMAE = 0.14923233393200308
8 Fold NMAE = 0.13952226729262457
9 Fold NMAE = 0.13377006722861914
10 Fold NMAE = 0.16896885834504044

 avg = 0.14751303016832956


In [361]:
xgb_pred = np.zeros(target.shape[0])
xgb_nmae = []
for i, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    xgb = XGBRegressor(random_state = 42, n_estimators = 2000, learning_rate = 0.03, max_depth = 5)
    xgb.fit(tr_x, tr_y, eval_set = [(val_x, val_y)], early_stopping_rounds = 300, verbose = 0)
    
    val_pred = np.expm1(xgb.predict(val_x))
    val_nmae = NMAE(np.expm1(val_y), val_pred)
    xgb_nmae.append(val_nmae)
    print(f'{i + 1} Fold NMAE = {val_nmae}')
    
    fold_pred = np.expm1(xgb.predict(target)) / kf.n_splits
    xgb_pred += fold_pred
print(f'\n avg = {np.mean(xgb_nmae)}')

1 Fold NMAE = 0.14821890018037917
2 Fold NMAE = 0.13982714044303307
3 Fold NMAE = 0.1522701369227135
4 Fold NMAE = 0.1429595116027757
5 Fold NMAE = 0.1304903376267611
6 Fold NMAE = 0.17381437717159687
7 Fold NMAE = 0.15762230336567912
8 Fold NMAE = 0.14785817601412035
9 Fold NMAE = 0.13724232287633986
10 Fold NMAE = 0.171118706158982

 avg = 0.15014219123623807


In [362]:
submission['Target'] = np.round((cb_pred + xgb_pred) / 2)

In [363]:
submission

,id,Target
0,1,10.0
1,2,14.0
2,3,5.0
3,4,10.0
4,5,9.0
...,...,...
2919,2920,4.0
2920,2921,7.0
2921,2922,10.0
2922,2923,9.0


In [364]:
submission.to_csv('cb_xgb_1.csv', index = False)